### 一. Executor框架
#### 1.1 任务的抽象 
1. Java类库中, 任务的抽象不是Thread, 而是Executor
2. Executor提供了一种标准方法, 将任务的提交和执行解耦开来; 其次还提供了对生命周期的支持, 以及统计信息, 应用程序管理机制和性能监视机制
3. Executor基于生产者-消费者模式, 提交任务的操作相当于生产者, 执行任务的线程属于消费者 (这也是各种Executor内部持有queue的原因)

#### 1.2 各种线程池
1. 线程池创建 api  
   线程池的创建和4个参数有关
     1. 线程池的初始大小: core size  
     2. 线程池的最大大小: maximum pool size  
     3. 线程最大可空闲时间: keep alive time
     4. 管理任务的队列: workQueue
    ```java
    public ThreadPoolExecutor(int corePoolSize,
                              int maximumPoolSize,
                              long keepAliveTime,
                              TimeUnit unit,
                              BlockingQueue<Runnable> workQueue)
    ```
2. 线程池创建逻辑  
    1. 线程池的初始大小, 也是线程池在没有任务执行时的大小, 只有在工作队列满了以后才会创建超出这个数量的线程
    2. 线程池的最大大小, 表示可同时活动的线程的最大数量  
    3. 线程最大空闲时间, 表示如果线程的空闲时间超过了这个限制, 且此时线程的数量大于线程池的初始大小, 则此线程被标记为可终止的
 
<img src="img/executors.png" width="80%">

3. 饱和策略    
   在ExectorService关闭后提交的任务,或是在线程池的有界队列被填满后, 将由"Regected Execution Handler"来处理, 他会抛弃任务
    1. AbortPolicy: 抛出异常
    2. DiscardPolicy: 默默丢弃任务
    3. DiscardOldestPolicy: 抛弃下一个要执行的任务, 并将自己插入到对垒
    4. CallerRunsPolicy: 在调用者线程中执行
<img src="img/reject.png" width="85%">

#### 1.3 CompletionService与BlockingQueue  
1. 如果向Exector提交了一组任务, 并且希望在计算之后获得这些任务的结果, 那么可以保存任务的Future, 然后反复调用timeout为0的get方法, 从而轮训判断任务是否完成; 这种方法有些繁琐, 类库CompletionService就是完成这个任务的  
2. CompletionService内部有一个存储结果的LinkedQueue; 任务是通过传进来的Executor执行的, 又复写了Futuetask的done方法, 让future执行完加入到内部的LinkedQueue中

<img src="img/completionservice.png" width="90%">

### 二. 任务的取消和关闭

#### 2.1 什么是正确的取消逻辑
1. 强行取消  
 无论线程执行到什么地方, 硬性的关闭线程退出; 我们很少希望这样的关闭方式, 因为这样会导致共享的数据结构处于不一致的状态.  
 Thread.stop和suspend方法就是这样, 被杀死的线程会默默地释放持有的锁, 造成对象的不可预知
2. 协作式取消   
 当需要停止时, 首先告诉线程应该停止了, 然后相应的线程发现需要停止后, 首先清除正在执行的工作, 然后再结束. 这种方式提供了更好的灵活性, 因为任务本身比发出取消请求的代码更知道如何执行清除工作
 
#### 2.2 中断Interrupt
1. interrupt就是协作式中断, 中断是取消的最合理方式  
2. 中断api   
    1. 检查线程是否被中断: Thread.currentThread().isInterrupted()
    2. 向线程发出中断请求: interrupt()
    3. 声明自己已相应过中断:  Thread.interrupted()
3. 3个api的逻辑   
 当有一个线程被执行interrupt()后, 该线程中的中断标记为设置为true, 此时若在线程中调用Thread.currentThread().isInterrupted()将会发挥true  
 正在线程响应中断后, 如果又调用了Thread.interrupted(), 则中断标记位会被重置为false, 以期待下次响应中断(如果此次中断后没退出的话)
 
4. 为什么大多数框架中, 当检测到了线程被interrupt, 会抛出interrupedExeption异常  
    1. 因为像Thread.sleep这样的方法, 在响应中断后, 往往会紧接着调用Thread.interrupted()重置标记位, 这样使得调用sleep方法的线程无法再得知该线程被中断了, 所以Thread.sleep中应该抛出中断异常通知上层调用者, 该线程被中断了;   
    2. 其次, 上层调用者除了使用Thread.currentThread().isInterrupted()检测中断外, 也应该设置一个boolean变量, 并在catch块中更改状态;  否则会遇到线程中断位重置问题. 如下所示, 线程的正确中断方式    
<img src="img/interrupt2.png">

#### 2.3 Executor中的取消
1. 取消某个任务  
    1. 当想要取消某个任务时, 不宜直接中断线程池, 因为并不知道当中断请求到达时正在运行什么任务, 因此只能通过任务的Future进行取消   
    2. 如果在future.get(timeout)时抛出异常, 此时也应该在finnaly代码块中取消任务
    <img src="img/futurecancell.png" width="60%">
    
2. 关闭整个ExecutorService
    1. shutdown: 等待队列中的所有任务完成后结束
    2. shutdownNow: 像所有worker线程发送interrupt, 并返回队列中未开启的任务(不包括已经开启但没运行结束的任务)
    ```java
    public interface ExecutorService extends Executor {
        void shutdown();
        List<Runnable> shutdownNow();
    }
    ```